In [1]:
import os
import pandas as pd
import sqlalchemy as db
from dotenv import load_dotenv

In [2]:
load_dotenv()

# config and credentials
server = os.getenv('server')
database = os.getenv('database')
username = os.getenv('username')
password = os.getenv('password')

# connection
engine = db.create_engine(
    'mssql://{}:{}@{}/{}?driver=ODBC+Driver+18+for+SQL+Server'.format(
        username, password, server, database
    )
)

# establish connection
connection = engine.connect()

### Wermers

In [4]:
query = '''
    DECLARE @RC int
    DECLARE @TimeFrame int
    DECLARE @Investor int

    EXECUTE @RC = [dbo].[Herding-Measures-Wermers] 
        {},
        {}
'''

investor = {
    1: '', 2: '-retail', 3: '-institutional',
}
timeframe = {
    1: '-daily', 2: '-weekly', 3: '-monthly', 4: '-quarterly', 5: '-yearly'
}

for tf, tf_desc in timeframe.items():
    for inv, inv_desc in investor.items():
        df = pd.read_sql_query(query.format(tf, inv), connection)
        df.to_csv('source/herding-wermers{}{}.csv'.format(inv_desc, tf_desc), index=False)
